**Types of Data Anomalies:** 

\- Update anomaly; when you have redundant data and only partially update the data (only updated addresses on some of the author's books)

\- Insertion Anomaly: can't insert data because it's missing other data in a row (Didn't have the author's address when adding a new book, can't add the book)

\- Deletion Anomaly: unintentionally lose data because we've deleted other data (We deleted the books, now the author has disappeared, even though we want to keep them on file)

**Avoiding Anomalies: Normalization Rules**

First Normal Form - Values in each column of a table must be atomic

Second Normal Form - All attributes not part of the key depend on the key

Third Normal Form - No transitive dependencies

\----

Normalized tables in a schema should show relations to the table 1:1, 1:N, ... 

**OLTP - Online Transaction Processing Systems**

\- many reads/writes

\- many processes

\- often normalized to third normal form 

**Analytical DBs**

\- data analysis

\- many reads by many processes

\- many writes by few processes 

\- Often Denormalized

**Denormalization** 

Can significantly improve performance

Why?

\- Data is often redundant

\- Contains non-atomic values

\- Tolerate transitive dependencies

\- Also has reduced risk of anomalies due to:

> \- few updates
> 
> \- batch inserts; data transformed before the inserts
> 
> \- streaming inserts; simple data structures
> 
> \- eliminate the need for complex joins

**Forms of Denormalized DBs**

\- Star Schema: Fact and Dimension tables

> \- Row-level orientation
> 
> \- Columnar data stores

\- Wide Column: everything in a table that is really wide

> \- The opposite of Tidy Data

**Partitioning**

**Vertical Partitioning**

\- Separating by column

\- Increases the number of rows in a data block with fewer columns

\- Global indexes for each partition

\- Can reduce I/O

  

**Horizontal Partitioning**  

\- Separation by subsets of rows \*most common

\- Limits scans to a subset of partitions by chunks of rows, same columns

\- Local indexes for each partition (smaller indexes)

\- Efficient adding and deleting 

  

**Range Partitioning -** type of horizontal

\- Parition on non-overlapping keys (like by month)

\- Parition by date is common

\- Numeric ranges are often used

\- Alphabetic, global region... 

\- _Partition on a value or list of values_

  

**Hash Partitioning**

\- Partition on modulus of hash of parition key

\- Pick the key based on the remainer of the modulus

\- Does not logically group into subgroups

\- _Want an even distribution of data across partitions_

In [1]:
-- Postgres, create a table with partitions and then their partitions

CREATE TABLE iot.sensor_msmt (
    sensor_id int NOT NULL, 
    msmt_date date NOT NULL, 
    temperature int, 
    humidity int)
    PARTITION BY RANGE (msmt_date);

-- It is partitioning by date, now we have to create the partitions
CREATE TABLE io_sensor_msmt_y2021m01 PARTITION OF iot.sensor_msmt
    FOR VALUES FROM ('2021_01_01') TO ('2021_01_31');

CREATE TABLE io_sensor_msmt_y2021m02 PARTITION OF iot.sensor_msmt
    FOR VALUES FROM ('2021_02_01') TO ('2021_02_28');

: schema "iot" does not exist

Total execution time: 00:00:00.005

**Materialized Views**

Persisited Results of a Query - a form a caching

  

\- Execute the query once

\- Save results once

\- Read many times

\- Trading space for time

  

**When to use materialized views?** 

\- Long-running queries

\- Complex queries, many joins

\- Computing aggregates or other derived data

\- Separate read and write operations

  

**When NOT to use Materialized Views?**

\- Eventual Consistency: you may not have the latest data in an updating system

\- Cost of update process

\- Concurrent reads during update? (Default in postgres)

\- Size of materialized view

\- Refresh Frequency

In [ ]:
-- Create a materialized view - kind of like a WITH clause

CREATE MATERIALIZED VIEW landon.mv_locations_expenses AS 
(
SELECT  l.hotel, 
        l.city, 
        l.state_province, 
        l.country, 
        e.year, 
        e.annual_payroll, 
        e.health_insurance, 
        e.supplies
FROM    
    landon.locations l
    LEFT JOIN
    landon.expenses e 
    ON (l.hotel_id = e.hotel_id)
)

SELECT * FROM landon.mv_locations_expenses;

-- IF there is an update, we can refresh the query

REFRESH MATERIALIZED VIEW landon.mv_locations_expenses;

**Read Replicas**

The primary server is responsible for all read and write queries at the same time

A **read replica** - data written to the primary is written to another instance of the data server

  

Using this, only WRITEs go to the primary server

Primary WRITEs to the Read Replica

All READ queries access the Read Replica  

\- Primary can focus on the writes

\- Multiple replicas can scale to meet read load

\- Especially useful when more read than write workload

\- Need to consider eventual consistency

**CHALLENGE**

An IoT company collects streaming data from thousands of sensors every minute

Data scientists will perform series analyses, including many aggregate data by sensor over hours and days

\- Low Latency is essential

\- Access to all data older than one hour

  

Design 

\- High-level model

\- What kind of structures

\- What kind of design patterns

\-------------------------------------------------------------------

  Solution: 

\- Sensor data written to table with the attributes for id and measures

\- Partition by time

\- Use Read Replicas to not bog down the server

\- Use Materialized Views, refresh once per hour for hourly

**B-Tree Indexing**
*Indexing for Analytical Queries*

- Reduces need to scan data blocks
- Comes at cost of additional writes during data loading
- The higher the cardinality of indexed data, the better the performance improvement
- Not used in analytical databases, such as Google BigQuery

---
Types of Indexes: 
- B-tree
- Bitmap
- Hash
- Special Purpose

## Balanced Tree
- Capture small amounts of data
- (Workhorse of indexing)
- Work well in many different cases
- Ability to look up values in logarithmic time
- Provides good order and lookup time

## Bitmap Indexes
- Used when there are a small number of possible values in a column (low cardinality)
- Filter by Bitwise operations such as AND, OR, NOT 
- Time to access is based on time to perform the Bitwise operations (fast)
- Read-intesive use cases with few writes (data warehouse/data science applications)

- Some databases allow you to explicity create bitmap indexes (Postgres does not)
- Postgres build bitmap indexes on the fly as needed

## Hash Functions
- Functions for mapping arbitrary length data to fixed-size string
- Has values virtually unique
- Even slight changes create a new hash
- Equal operations only
- Can be smaller than B-Tree indexes
- Comparable with speed of B-Tree build and access

## GiST - Generalized Search Tree 
- Balance Tree-structure access method
- Used as template to implement other indexing schemes
    - B-Tree -- self-balancing tree; operations in log time
    - R-Tree -- index of multidimensional data such as geographical coordinates
- Used in Postgres for indexing: 
    - hstore
    - ltree

--- Operator Classes and Indexed Data Types
- Box_ops  = box
- Circle_ops = circle
- Index_ops - inet, cidr
- Point_ops - point
- Poly_ops - polygon
- Range_ops - any range type
- Tsquery_ops - text queries
- Tsvectory_ops - tsvectors, sorted list of distinct lexemes

## SP-GiST - Space Partitioned GiST
- Supports partitioned search trees
- Useful for non-balanced data structures
    - quadtree - tree with internal nodes having 4 children
    - k-d tree -- k-dimensional tree, used to index point in k-dimensional space
- Develop custom indexes

--- Operators 
- kd_point_ops
- quad_point_ops
- range_ops -- any range type
- box_ops
- poly_ops
- text_ops
- Inet_ops -- inet, cidr

## GIN and BRIN Indexes
- Generalized Inverted Index
- Used when data to be indexed are composite values
- Composite values require index to search for elements within composite item
- EX:  Words in a document (each word is an individual element that can be indexed)

Index stores data in pairs (key, posting list)
- A key is an element value
- Posting list is a set of row IDs in which the key occurs

- Access methods defines when creating a GIN index based on the types of data indexed

Built-In Operator Classes
- Array_ops - any array
- Json_ops - jsonb
- Json_path - jsonb
- Tsvector_ops - text_vectors

Tips 
- Insertion can be slow because many keys may be inserted for each item (many words in a document)
- For very large bulk operations, likely faster to drop and recreate index
- Postgres can postpone much of indexing workk by using temporary lists
    - Temp lists eventually inserted into index using optimized bulk insertion techniques
    - Disadvantage: temp list must ALSO be searched in addition to regular index when both indexed
    - Large temp lists will slow Searches significantly
    - Tradeoff: Slow search vs Slow load
    - Can Disable **fastupdate** parameter in **CREATE INDEX** to disable temporary lists

## BRIN Index
- Block Range Index
- Used with very large tables 
- Column data has a correlation with physical data (postal code, coordinates)

- Block ranges are pages that are physically adjacent in a table
- BRIN indexes store summary info about block ranges
- BRIN indexes tend to be small
    - Entries are for entire block ranges, not indiv elements
    - Quickly scan, skip large segments of a table when searching (min/max)

Operators: 
- Date_minmax_ops
- Char_minmax_ops
- Float8_minmax_ops
- Timestamp_minmax_ops
- UUID_minmax_ops
- Many more ...



---
**CHALLENGE:**
 - You have a very large dataset of insurance claim details, and you want to ingest data into existing db 
 - Each claim has a unique id and 12 columns of data about the claim
 - Existing db has a table of all claim numbers ever generated

 How would you index the new claim detail data to optimize a join operation on the claim ID? 
 - B-Tree
 - Hash

 SOLUTION: 
 - Because using claim IDs, 1 row in each table will have the claim id, use a Hash Index - converts the ID to 32 bit integer


\---

**Query Optimization**  

  

SQL Figures out the procedural steps and executes for us: 

  

1\. Scanning Tables - check rows in column to check filter, fetch the data block containing the row, then return the row

> \- This only works with tables holding setups in rows, not columnar 
> 
> \- Smaller tables are ideal, larger tables are less efficient, unless they only need to be scanned once
> 
> \- Larger tables when queried repeatedly are better suited for using indexes. 

2\. Indexes - adds an order to the table, filters by checking the index for a match, then retrieves the row

> \- Imagine looking for last names starting with 'n'. Scanning requires going through the entire table
> 
> \- Indexing would only scan indexes in that alphabetical range, once it goes from M to P, the query can stop

  

Joining Tables - How do you match rows? 

> How do you match foreign and primary keys? 
> 
> 1\. Nested Loop Join - compare all rows in both tables to each other

> > \- Loop through one table for each row
> > 
> > \- Loop through other table
> > 
> > \- At each step, compare keys
> > 
> > \- Simple to implement
> > 
> > \- Can be expensive
> > 
> >   

> 2\. Hash Join - calculate hash values of both keys, join by matching hash values

> > \- Compute hash values of key values in smaller table
> > 
> > \- Store in hash table, which has hash values and row attributes
> > 
> > \- Scan larger table; find rows from smaller hash table

> 3\. Sort Merge Join - sort both tables, then join rows taking advantage or order

> > \- Sort both tables
> > 
> > \- Compare rows like nested loop join, but stop when no longer possible to find a match due to sort order
> > 
> > \- Scans the driving table only once

  

**Analyze**

> **\-** Plan builder relies on stats about data in tables
> 
> \- Usually stats are kept up to date (autovacuum)
> 
> \- Sometimes stats get out of date
> 
> \- ANALYZE command updates statistics

In [ ]:
-- This creates the iot schema and then two tables 

-- CREATE SCHEMA iot;

DROP TABLE IF EXISTS iot.sensor_msmt;
CREATE TABLE iot.sensor_msmt (
   sensor_id        int not null,
   msmt_date        date not null,
   temperature      int,
   humidity         int
) PARTITION BY RANGE (msmt_date);


CREATE TABLE iot.sensor_msmtt_y2021m01 PARTITION OF iot.sensor_msmt
    FOR VALUES FROM ('2021-01-01') TO ('2021-02-01');


CREATE TABLE iot.sensor_msmtt_y2021m02 PARTITION OF iot.sensor_msmt
    FOR VALUES FROM ('2021-02-01') TO ('2021-03-01');


This next code basically does a cross join without formally declaring a join.

In [5]:
SELECT * 
FROM 
    (SELECT * FROM generate_series(1,100)) AS t1, 
    (SELECT * FROM generate_series('2021-01-01 00:00'::timestamp, 
                        '2021-02-15 00:00'::timestamp,
                        '1 minutes')) AS t2


(6480100 row(s) affected)

Displaying Top 5000 rows.

Total execution time: 00:00:29.805

generate_series,generate_series
1,2021-01-01T00:00:00
1,2021-01-01T00:01:00
1,2021-01-01T00:02:00
1,2021-01-01T00:03:00
1,2021-01-01T00:04:00
1,2021-01-01T00:05:00
1,2021-01-01T00:06:00
1,2021-01-01T00:07:00
1,2021-01-01T00:08:00
1,2021-01-01T00:09:00


Next, we can create a CTE for the above query, and then add humidity sensor data. After checking that it worked, adding the insert function to put it into the table.

In [8]:
INSERT INTO iot.sensor_msmt 
(
    WITH sensors_datetimes AS 
    (
        SELECT * 
        FROM 
            (SELECT * FROM generate_series(1,100)) AS t1, 
            (SELECT * FROM generate_series('2021-01-01 00:00'::timestamp, 
                                '2021-02-15 00:00'::timestamp,
                                '1 minutes')) AS t2
    )
    SELECT sd.*, 
        floor(random()*30) AS temperature, 
        floor(random()*80) AS humidity
    FROM sensors_datetimes sd
)

Commands completed successfully

Total execution time: 00:00:11.942

Next we can use the EXPLAIN SELECT.

We can do this before and after using indexes to see if implementing the index is more efficient than not using them.

In [13]:
EXPLAIN SELECT *
FROM 
    iot.sensor_msmt
WHERE 
    sensor_id BETWEEN 10 AND 20;

CREATE INDEX idx_sensor_msmt_id ON iot.sensor_msmt(sensor_id);


(5 row(s) affected)

Commands completed successfully

Total execution time: 00:00:01.727

QUERY PLAN
Append (cost=0.00..135758.20 rows=705741 width=16)
-> Seq Scan on sensor_msmtt_y2021m01 sensor_msmt_1 (cost=0.00..91090.00 rows=486725 width=16)
Filter: ((sensor_id >= 10) AND (sensor_id <= 20))
-> Seq Scan on sensor_msmtt_y2021m02 sensor_msmt_2 (cost=0.00..41139.50 rows=219016 width=16)
Filter: ((sensor_id >= 10) AND (sensor_id <= 20))


In [14]:
EXPLAIN SELECT *
FROM 
    iot.sensor_msmt
WHERE 
    sensor_id BETWEEN 10 AND 20;

(5 row(s) affected)

Total execution time: 00:00:00.003

QUERY PLAN
Append (cost=0.43..23865.39 rows=705741 width=16)
-> Index Scan using sensor_msmtt_y2021m01_sensor_id_idx on sensor_msmtt_y2021m01 sensor_msmt_1 (cost=0.43..14024.93 rows=486725 width=16)
Index Cond: ((sensor_id >= 10) AND (sensor_id <= 20))
-> Index Scan using sensor_msmtt_y2021m02_sensor_id_idx on sensor_msmtt_y2021m02 sensor_msmt_2 (cost=0.43..6311.75 rows=219016 width=16)
Index Cond: ((sensor_id >= 10) AND (sensor_id <= 20))


So the cost with the indexes is much lower than the cost without using indexes.

In [15]:
CREATE TABLE iot.sensors AS
(
    WITH sensor_ids AS 
        (SELECT i FROM generate_series(1,100) AS i)
    SELECT 
        i as id, 'Sensor' || i::text AS sensor_name
    FROM 
        sensor_ids
)

Commands completed successfully

Total execution time: 00:00:00.013

We now have a table we can join to. We want to get a sensor name from the sensor table, and then the sensors and date from the sensors table.

In [16]:
EXPLAIN SELECT
    s.sensor_name, 
    sm.msmt_date, 
    sm.temperature, 
    sm.humidity
FROM 
    iot.sensor_msmt AS sm
LEFT JOIN 
    iot.sensors AS s
ON 
    sm.sensor_id = s.id

(7 row(s) affected)

Total execution time: 00:00:00.012

QUERY PLAN
Hash Left Join (cost=3.25..188933.63 rows=6480100 width=20)
Hash Cond: (sm.sensor_id = s.id)
-> Append (cost=0.00..132229.50 rows=6480100 width=16)
-> Seq Scan on sensor_msmtt_y2021m01 sm_1 (cost=0.00..68770.00 rows=4464000 width=16)
-> Seq Scan on sensor_msmtt_y2021m02 sm_2 (cost=0.00..31059.00 rows=2016100 width=16)
-> Hash (cost=2.00..2.00 rows=100 width=12)
-> Seq Scan on sensors s (cost=0.00..2.00 rows=100 width=12)


This automatically performed a hash join, but in the first append, there are 2 sequence scans. Next we'll make the data a little smaller just to see how it adapts.

In [17]:
EXPLAIN SELECT
    s.sensor_name, 
    sm.msmt_date, 
    sm.temperature, 
    sm.humidity
FROM 
    iot.sensor_msmt AS sm
LEFT JOIN 
    iot.sensors AS s
ON 
    sm.sensor_id = s.id
WHERE 
    s.id = 30

(8 row(s) affected)

Total execution time: 00:00:00.001

QUERY PLAN
Nested Loop (cost=0.43..2619.61 rows=63154 width=20)
-> Seq Scan on sensors s (cost=0.00..2.25 rows=1 width=12)
Filter: (id = 30)
-> Append (cost=0.43..1985.83 rows=63154 width=16)
-> Index Scan using sensor_msmtt_y2021m01_sensor_id_idx on sensor_msmtt_y2021m01 sm_1 (cost=0.43..1150.40 rows=43598 width=16)
Index Cond: (sensor_id = 30)
-> Index Scan using sensor_msmtt_y2021m02_sensor_id_idx on sensor_msmtt_y2021m02 sm_2 (cost=0.43..519.66 rows=19556 width=16)
Index Cond: (sensor_id = 30)


It no longer uses the hash join, it is using a nested loop (which works better for smaller amounts of data)  

  

\---

**Challenge:** 

A colleague has asked for your help in analyzing a long-running query: 

> \- The query uses 3 tables and 2 left joins

> \- Two tables have over 500,000 rows, the third has 200 rows

> \- How would you proceed to improve performance?

  

  

My solution: Hash values - create hash values on the smallest table, then apply to the larger tables. To find out, use an EXPLAIN....

> \- Use EXPLAIN to view query executino plan 
> 
> \- Look for full scans and indexing opportunities
> 
> \- Assess how indexes are used with joins
> 
> \- Look for opportunities to filter the dataset
> 
> \- Run ANALYZE to ensure statistics are up to date

**USER-DEFINED FUNCTIONS!**

- SQL provides many functions for operating on data
    - Aggregates
    - String manipulation
    - Pattern matching
    - Date and time
    - Geometric
- Data science and analytics need more functions!

  

**Custom Functions: Five Types of User-Defined Functions**

1\. Query language functions written in SQL

2\. Procedural functions written in PL/pgSQL

3\. Internal functions written in C and statically linked to PostgreSQL 

4\. C language functions available through shared libraries

5\. PL/Python by extension - not a trusted langauge

  

  

\---

  

**Query Language Functions**

Execute arbitrary list of SQL statements

Return result of last statement

- Last statment must be SELECT
- Or the function must be declared VOID
- May be declared as a return set

  

**Types of SQL Statements Available:** 

- SELECT
- INSERT
- UPDATE
- DELETE
- CREATE

**CANNOT** use transactional clauses

- BEGIN
- ROLLBACK
- COMMIT
- SAVEPOINT

\---

**CREATE FUNCTION** 

is the syntax to create a function...

There is also the option for **CREATE OR REPLACE FUNCTION**

**DROP FUNCTION** 

> **\-** Deletes the existing function
> 
> \- Creates a new function
> 
> \- Need to drop all existing rules, views, triggers, etc that refer to the dropped function.

Let's create a function for the Harmonic Mean: 2xy/(x + y)

  

It's the reciprocal of an arithmetic mean, and can give an average rate. 

Also used in information retrieval in ML - called the F1 score

In [ ]:
-- The Hard Way

SELECT 
    bed_occupancy_rate, 
    room_occupancy_rate, 
    round(((2 * bed_occupancy_rate * room_occupancy_rate) / 
        (bed_occupancy_rate + room_occupancy_rate))::NUMERIC,2) AS composite_occupancy_rate
FROM 
    occupancy_rates

The definition of the function is captured between the sets of $$

The last defines the language as SQL

In [18]:
-- Using Custom Functions
-- This Can be CREATE FUNCTION or CREATE OR REPLACE FUNCTION

CREATE FUNCTION harmonic_mean (x NUMERIC, y NUMERIC) 
RETURNS NUMERIC
AS 
$$
    SELECT ROUND(((2 * x *  y)/ (x + y))::NUMERIC, 2)
$$
LANGUAGE SQL;

Commands completed successfully

Total execution time: 00:00:00.007

In [19]:
SELECT harmonic_mean(2,7)


(1 row(s) affected)

Total execution time: 00:00:00.015

harmonic_mean
3.11


**Function Overloading**

A single function can have multiple definitions

Different definitions require different parameter signatures

- CREATE FUNCTION harmonic\_mean (x NUMERIC, y NUMERIC) RETURNS NUMERIC
- CREATE FUNCTION harmonic\_mean (x TEXT, y TEXT) RETURNS NUMERIC

Can be useful when it can be applied to different data types - avoids ambiguities like entering 8, which can be INT, SMALLINT, or TEXT

The ::NUMERIC converts the entry (if possible) into a numeric data type

In [20]:
CREATE OR REPLACE FUNCTION harmonic_mean (x NUMERIC, y NUMERIC) 
RETURNS NUMERIC
AS 
$$
    SELECT ROUND(((2 * x *  y)/ (x + y))::NUMERIC, 2)
$$
LANGUAGE SQL;

CREATE OR REPLACE FUNCTION harmonic_mean (x TEXT, y TEXT) 
RETURNS NUMERIC
AS 
$$
    SELECT ROUND(((2 * x::NUMERIC *  y::NUMERIC)/ (x::NUMERIC + y::NUMERIC))::NUMERIC, 2)
$$
LANGUAGE SQL;

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.013

In [21]:
SELECT harmonic_mean(2.2, 7.1), harmonic_mean('2.2', '7.1')

(1 row(s) affected)

Total execution time: 00:00:00.013

harmonic_mean,harmonic_mean
3.36,3.36


**Function Volatility**

- Functions have a volatility classification
- Specifies the kinds of behaviors that a function has
- Used to allow the optimizer to make assumptions about the function

**Classifications**

1. **Volatile** (default)
1. Can perform any operation, including making changes to the db
2. Optimizer makes no assumptions about the function
3. Function is reevaluated at every row the function is needed
4. Use it if you have to, use others if you can
3. **Stable** 
1. Cannot modify the db
2. Guaranteed to return same results given the same args for all rows within a single statement
3. Enables some optimization
4. Better than volatile
5. **Immutable**
1. Cannot modify db
2. Guaranteed to return same results given the same args in ALL cases
3. Enables the most optimizations
4. Most restrictive

In [ ]:
CREATE OR REPLACE FUNCTION harmonic_mean (x NUMERIC, y NUMERIC) 
RETURNS NUMERIC
AS 
$$
    SELECT ROUND(((2 * x *  y)/ (x + y))::NUMERIC, 2)
$$
LANGUAGE SQL IMMUTABLE;

**Key Points about Volatility**

- Use the most strict volatility classification possible for optimal performance
- Any function with side effects must be VOLATILE
- If the function has no side-effects but uses another function that can change value in a query, the user defined function must be VOLATILE

**PYTHON Functions**

- Can write PostgreSQL functions in Python
- Uses a postgres extention (PL/Python)

PROS: 

- Language is easy to use
- Reuse codfe already written
- Avoid having to learn PL/pgSQL for procedural code

CONS: 

- Not a trusted language
- Code can damage the db
- Installation can vary by postgres package

**Installation**

- Needs superuser privaleges 
    - ALTER USER Postgres WITH superuser
- CREATE EXTENSION plpython3u
- After install, other can write pl/python functions
- Arguments are treated as **Global Variables**

In [ ]:
CREATE FUNCTION pymax (x x INTEGER, y INTEGER)
    RETURNS INTEGER
AS 
$$
    if x>y:
        return x
$$ LANGUAGE plpython3u;

**Challenge**

Write a user-defined SQL function that Returns a TRUE if the input is a palindrome

In [23]:
CREATE OR REPLACE FUNCTION is_palindrome (str TEXT) 
    RETURNS BOOLEAN
AS 
$$
    SELECT reverse(str) = str
$$ LANGUAGE SQL IMMUTABLE;

SELECT 
    is_palindrome('evilolive') AS evilolive, 
    is_palindrome('fascist') AS fascist;



Commands completed successfully

(1 row(s) affected)

Total execution time: 00:00:00.009

evilolive,fascist
True,False


**Federated Queries**

Utilizes SQL/MED - management of external data

- Foreign Data Wrappers (FDW)
- Allow for viewing data outside of the db
- Datalinks provide db functionality

**Create an Extension and Server**

CREATE EXTENSION postgres\_fdw

CREATE SERVER external\_db\_server

> FOREIGN DATA WRAPPER postgres\_fdw
> 
> OPTIONS (host 'mydb.example.com', dbname, 'data\_warehouse')

  

**Map Users**

CREATE USER MAPPING FOR CURRENT\_USER

> SERVER external\_db\_server
> 
> OPTIONS (user 'analyst', password 'dasoinesogoidsn')

  

CREATE SCHEMA external\_sales;

IMPORT FOREIGN SCHEMA sales

> FROM SERVER external\_db\_server
> 
> INTO external\_sales;

  

CREATE FOREIGN TABLE page\_visit\_log (

> log\_time timestamp, 
> 
> user\_name text, 
> 
> web\_page text

<span style="background-color: rgba(127, 127, 127, 0.1);">) SERVER log_data</span>  
OPTIONS (filename 'app/ecommerce/logs page\_visits.log', format 'csv');  

>

**Bloom Filter Indexes**

- Space-efficient method for deetermining set membership
- Useful for quickly finding sets with values
- **Lossy representation of data -** can't take compressed version and recreate
- **Probabilistic Data Structure** 
    - may produce false positive
    - Never produces false negatives
- Accuracy is a function of the number of bits used in the filter
- About 10 bits per element will yield a 1% false positive rate

These are useful when: 

- Table has many attributes
- Queries filter on many different combinations of attributes
- B-Tree Indexes are faster but may require many indexes

LIMITATIONS: 

- Bloom filter indexes only support equality operator on int4 and text data types
- B-Tree indexes support the above

In [ ]:
CREATE EXTENSION bloom
CREATE INDEX idx_locations_blm ON locations
    USING bloom (city, state_province, country);

CREATE INDEX idx_locations_blm ON locations
    USING bloom (city, state_province, country)
    WITH (length = 128);


**Hstore for Key-Value pairs**

Data type for storing k-v pair sets

Columns defined as hstore data type

- CREATE EXTENSION hstore
- CREATE TABLE my\_kv\_table (id serial, primary key kv\_set hstore)

Key-Value Pairs represented as a string: 

- 'key1 =\> value1, key2 =\> value2, key3 =\> value3'

**Why Use them?**

- When keys are not predefined
- Large number of attributes that are not used
    - Do not need to change table definition to accommodate new columns
- GIN and GIST indexes can index all keys
- Example - a catalog

**Limitations**

- All keys and values are stored as strings
- Do not support hierarchical data structures (JSONB and XML can do this)

In [26]:
-- CREATE EXTENSION hstore;

CREATE TABLE books (
    id serial PRIMARY KEY, 
    title text, 
    attributes hstore
); 





Commands completed successfully

Total execution time: 00:00:00.012

In [29]:
INSERT INTO books (title, attributes) VALUES
    ('SQL for Data Science', 
    'language=>English, page_cnt=>500, pub_year=>2021'
    );

INSERT INTO books (title, attributes) VALUES
    ('SQL for Data Science 2', 
    'language=>English, page_cnt=>600, pub_year=>2022'
    );

SELECT * FROM books;


Commands completed successfully

Commands completed successfully

(4 row(s) affected)

Total execution time: 00:00:00.014

id,title,attributes
1,SQL for Data Science,"""language""=>""English"", ""page_cnt""=>""500"", ""pub_year""=>""2021"""
2,SQL for Data Science 2,"""language""=>""English"", ""page_cnt""=>""600"", ""pub_year""=>""2022"""
3,SQL for Data Science,"""language""=>""English"", ""page_cnt""=>""500"", ""pub_year""=>""2021"""
4,SQL for Data Science 2,"""language""=>""English"", ""page_cnt""=>""600"", ""pub_year""=>""2022"""


In [30]:
SELECT title, attributes->'page_cnt' FROM books;


(4 row(s) affected)

Total execution time: 00:00:00.003

title,Column2
SQL for Data Science,500
SQL for Data Science 2,600
SQL for Data Science,500
SQL for Data Science 2,600


**JSON and JSONB for semi-strutured data**

**Relationa and NoSQL (The B stands for Better)**

- Modeling document databases
- Flexible schema
- Nested structures
- Query and index support for document structures

Validate a string is valid JSON

Allows querying and indexing on elements in JSON

JSONB - binary representation; more efficient storage

  

**JSONB**

- Supports nested JSON structures
- Uses GIN indexes
- Indexes all key-value pairs in structure

In [31]:
CREATE TABLE customer_summary (
    id serial PRIMARY KEY, 
    customer_doc jsonb);

INSERT INTO customer_summary (customer_doc) VALUES
    ('{"customer_name":{"first_name":"Alice",
                        "last_name":"Johnson"},
        "address":{"Street": "76532 Port Path", 
                    "City": "Boston", 
                    "State": "MA"}, 
        "purchase_history":{"annual_purchase_value":[100, 200, 350],
        "lifetime_value":1500}
        }')

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.017

In [35]:
SELECT 
    customer_doc -> 'customer_name'->>'first_name'

FROM 
    customer_summary


(1 row(s) affected)

Total execution time: 00:00:00.002

Column1
Alice


**Hierarchical Data and L-Trees**

- Typically trying to follow paths from one leaf to the next
    - Start with the parent node, then the id of the nodes below along the path
- Usually use recursive commont table expressions
- (In oracle, you can use CONNECT BY)

**Materialized Paths** 

- Each node in tree is in its own row
- Each row representing a node has a path column
- Path column contains string of ancestors
- Search and pattern match string

The Ltree extension allows us to not have to go through every row repeatedly to look for matching paths

  

**Ltree**

- PostgreSQL extension for working with trees and paths
- Many operators for operating and comparing baths
- supports GiST and B-Tree indexes

LTree Operators: 

- @\>  - ancestor
- \<@  - descendent
- ||  - concatenate tree paths
- ~  - does ltree match an ltree text query? 
- Many more... consult the documentation

In [43]:
-- CREATE EXTENSION ltree; 

CREATE TABLE paths_to_nodes(
    id serial PRIMARY KEY, 
    node text, 
    path ltree);

CREATE INDEX idx_paths_to_nodes ON paths_to_nodes USING gist(path);

insert into  paths_to_nodes (node, path) values ('A', 'A');
insert into  paths_to_nodes (node, path) values ('B', 'A.B');
insert into  paths_to_nodes (node, path) values ('C', 'A.C');
insert into  paths_to_nodes (node, path) values ('D', 'A.B.D');
insert into  paths_to_nodes (node, path) values ('E', 'A.B.E');
insert into  paths_to_nodes (node, path) values ('F', 'A.C.F');
insert into  paths_to_nodes (node, path) values ('G', 'A.C.G');
insert into  paths_to_nodes (node, path) values ('H', 'A.B.D.H');
insert into  paths_to_nodes (node, path) values ('I', 'A.B.D.I');
insert into  paths_to_nodes (node, path) values ('J', 'A.B.D.J');												 
insert into  paths_to_nodes (node, path) values ('K', 'A.C.F.K');

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

...

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.011

In [44]:
SELECT * FROM paths_to_nodes

(11 row(s) affected)

Total execution time: 00:00:00.003

id,node,path
1,A,A
2,B,A.B
3,C,A.C
4,D,A.B.D
5,E,A.B.E
6,F,A.C.F
7,G,A.C.G
8,H,A.B.D.H
9,I,A.B.D.I
10,J,A.B.D.J


Find all descendents with a A.B at the front of the path

In [45]:
SELECT * 
FROM paths_to_nodes
WHERE 'A.B' @> path

(6 row(s) affected)

Total execution time: 00:00:00.014

id,node,path
2,B,A.B
4,D,A.B.D
5,E,A.B.E
8,H,A.B.D.H
9,I,A.B.D.I
10,J,A.B.D.J


Now find anything that has E in the path

In [47]:
SELECT * 
FROM paths_to_nodes
WHERE '*.C.*' ~ path

(4 row(s) affected)

Total execution time: 00:00:00.002

id,node,path
3,C,A.C
6,F,A.C.F
7,G,A.C.G
11,K,A.C.F.K


We can also specify that we only want paths where there is always one node after node B

In [48]:
SELECT * 
FROM paths_to_nodes
WHERE '*.B.*{1}' ~ path

(2 row(s) affected)

Total execution time: 00:00:00.001

id,node,path
4,D,A.B.D
5,E,A.B.E


Now show at least 0 and up to 1 node in the path

In [50]:
SELECT * 
FROM paths_to_nodes
WHERE '*.B.*{0,1}' ~ path

(3 row(s) affected)

Total execution time: 00:00:00.001

id,node,path
2,B,A.B
4,D,A.B.D
5,E,A.B.E


In [51]:
WITH paths_to_concat AS 
(
    SELECT 
        *
    FROM 
        paths_to_nodes p2
    WHERE 
        path ~ '*.C.*'
)
SELECT p1.id, p1.node, p1.path, p1.path || p2.path -- This || concatenates p1 and p2 paths
FROM 
    paths_to_nodes p1,
    paths_to_nodes p2
WHERE 
    p1.path ~ '*.B.*'


(66 row(s) affected)

Total execution time: 00:00:00.014

id,node,path,Column4
2,B,A.B,A.B.A
4,D,A.B.D,A.B.D.A
5,E,A.B.E,A.B.E.A
8,H,A.B.D.H,A.B.D.H.A
9,I,A.B.D.I,A.B.D.I.A
10,J,A.B.D.J,A.B.D.J.A
2,B,A.B,A.B.A.B
4,D,A.B.D,A.B.D.A.B
5,E,A.B.E,A.B.E.A.B
8,H,A.B.D.H,A.B.D.H.A.B


Column 4 is actually an ltree, not a string

  

**Challenge**

- Ecommerce company catalog of products
- Different types of products
- May query and filter on any attribute
- Performance is a key consideration
- How would you design a table to support products? 

  

**Solution:** 

- Semi-structured data
- Need indexes to support query performance
- Could use columns for each attribute but not practical to maintain --\>
- Use JSONB column to provide document structure and indexing